In [ ]:
import polars as pl
import pandas as pd
import networkx as nx
import panel as pn
#import tracemalloc
import os
#tracemalloc.start()
import rtsvg
rt = rtsvg.RACETrack()

#
# Select By Color doesn't work if there's a cloud with multiple
# colors within it...  if the cloud is only single color (which
# is currently rendered as "white"), it works...
#
# If multiple clouds are mixed colors, then selecting by color
# for them selects the multi-colored clouds...
#
# Ctrl-Click is an issue on Mac when using "_igl_.show()"... it
# popups up the popup menu... one (poor) solution is to start
# your drag with the mouse and then press/hold the "ctrl" key...
#
# ... this doesn't necessarily help with some of the other
# "ctrl"-key issues...
#
# This problem isn't apparent when using panel in VSCode...
# ... however VSCode will eventually die (unsure why) when you
# use the interactiveGraphLayout() for a long time...  it doesn't
# appear to be a memory issue...  The way this looks is that all
# the non-cell areas in VSCode turn gray...  (when using the dark 
# theme)
#

edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.rt_layout.parquet'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df            = pl.DataFrame(_lu_)
df            = df.with_columns(pl.col('fm').cast(pl.String), pl.col('to').cast(pl.String))
g             = rt.createNetworkXGraph(df, [('fm','to')])
_pos_         = nx.spring_layout(g)
_relates_     = [('fm','to')]
_node_colors_ = {}
community_i   = 0
for _community_ in nx.community.louvain_communities(g):
    community_i += 1
    for _node_ in _community_: _node_colors_[_node_] = rt.co_mgr.getColor(community_i)
_ln_params_ = {'relationships':_relates_, 'pos':_pos_, 'color_by':'gp', 'node_color':_node_colors_, 'link_shape':'curve', 'link_size':'vary', 'draw_labels':False}
_l_  = rt.link    (df, track_state=True, **_ln_params_)
_ln_ = rt.linkNode(df, track_state=True, **_ln_params_)
_l_._repr_svg_(), _ln_._repr_svg_()
#rt.tile([_l_, _ln_], spacer=10)
_igl_ = rt.interactiveGraphLayout(df, _ln_params_, w=600, h=600)
_igl_

In [ ]:
#current, peak = tracemalloc.get_traced_memory()
#print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

In [ ]:
_ln_params_ = {'relationships':_relates_, 'pos':{}, 'color_by':'gp', 'node_color':_node_colors_, 'draw_labels':False}
_igl2_ = rt.interactiveGraphLayout(df, _ln_params_, w=600, h=600, use_linknode=True)
_histogram_ = rt.interactivePanel(df, {(0,0,1,1):('histogram', {'bin_by':'fm', 'count_by':'to'})}, 200, 600)
_igl_.register_companion_viz(_histogram_),     _igl_.register_companion_viz(_igl2_)
_igl2_.register_companion_viz(_histogram_),    _igl2_.register_companion_viz(_igl_)
_histogram_.register_companion_viz(_igl_),     _histogram_.register_companion_viz(_igl2_)
#pn.Row(_igl_, _igl2_, _histogram_).show()

In [ ]:
# Simple test for link labels
df = pl.DataFrame({'fm':'a b c d'.split(), 
                   'to':'b c d a'.split(),
                   't': '1 1 2 2'.split()})
_params_ = {'df':df, 'relationships':[('fm','to')], 'pos':{'a':(0,0),'b':(1,0), 'c':(1,1), 'd':(0,1)}, 'bounds_percent':.3,
            'draw_labels':True, 'node_labels':{'b':'Label For B\nMore'},
            'link_labels':True, 'color_by':'t', 'link_color':'vary', 'link_size':2}
#rt.tile([rt.linkNode(**_params_),rt.link(**_params_)], spacer=10)

In [ ]:
a = ['a','b','c','d']
a.index('b')